<a href="https://colab.research.google.com/github/i-am-akash/Pyspark_Practice/blob/main/calculateAverage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Calculate Min and Max of movie ratings



In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pyspark import SparkConf, SparkContext
Conf = SparkConf().setAppName("Finding Min and max for movie ratings")
sc = SparkContext.getOrCreate(conf = Conf)

In [7]:
csv_File = sc.textFile("/content/sample_data/average_quiz_sample.csv")
csv_File.collect()

['JAN,NY,3.0',
 'JAN,PA,1.0',
 'JAN,NJ,2.0',
 'JAN,CT,4.0',
 'FEB,PA,1.0',
 'FEB,NJ,1.0',
 'FEB,NY,2.0',
 'FEB,VT,1.0',
 'MAR,NJ,2.0',
 'MAR,NY,1.0',
 'MAR,VT,2.0',
 'MAR,PA,3.0']